In [3]:
import numpy as np
from tqdm.notebook import tqdm
import trimesh
from queue import PriorityQueue
import matplotlib.pyplot as plt
from make_vtk import write_lines_to_vtk,write_points_to_vtk
from vtk_revise import read_vtk,write_vtk
from faster_generate_cone import generate_vectors_in_cone,plot_vectors_in_3d
from generate_interpolate_vector import generate_multiple_vectors

In [4]:
def normalize_points(points):
    """Normalize a set of 3D points."""
    norms = np.linalg.norm(points, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Avoid division by zero
    return points / norms

In [5]:
white = read_vtk('./AllCortexData/GS_CortexODE/SUBJ_001_MR_BL/surf/lh.CortexODE.white.vtk')
pial = read_vtk('./AllCortexData/GS_CortexODE/SUBJ_001_MR_BL/surf/lh.CortexODE.pial.vtk')

white_vertices = white['vertices']
white_faces = white['faces'][:,1:]

pial_vertices = pial['vertices']
pial_faces = pial['faces'][:,1:]

In [6]:
# Load your target mesh
origin_mesh = trimesh.Trimesh(vertices=white_vertices, faces=white_faces)
target_mesh = trimesh.Trimesh(vertices=pial_vertices, faces=pial_faces)


white2pial_dir = normalize_points(pial_vertices - white_vertices)

locations, index_ray, index_tri = target_mesh.ray.intersects_location(ray_origins=white_vertices, ray_directions=white2pial_dir)

In [7]:
pairs = []
thicknesses = []
not_intersect = []

In [9]:
for idx in tqdm(range(len(white_vertices))):
    real_idx = np.where(index_ray == idx)
    subset_locations = locations[real_idx]
    subset_origin = white_vertices[idx]
    distances = [np.linalg.norm(loc - subset_origin) for loc in subset_locations]
    
    if distances:
        min_distance_index = np.argmin(distances)
        closest_location = subset_locations[min_distance_index]
        closest_distance = distances[min_distance_index]
        
        pairs.append((subset_origin, closest_location))
        thicknesses.append(closest_distance)
        
    else:
        thicknesses.append(float('nan'))
        not_intersect.append(subset_origin)
    

  0%|          | 0/142422 [00:00<?, ?it/s]

In [10]:
print(len(not_intersect))

1320
